In [1]:
import os
os.chdir('../../')
!pwd

/root/python/myenv/medical-coding-reproducibility-main


In [2]:
import logging
import math
import os
from pathlib import Path
import json 
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel,Field
from uvicorn import Config, Server
from typing import List
import pandas as pd
import gc
from src.utils.fun_retrieval import prf

from src.settings import (
    DOWNLOAD_DIRECTORY_MIMICIV
)


from prepare_data.utils import (
    TextPreprocessor,
    preprocess_documents,
    load_gz_file_into_df,
    ID_COLUMN, SUBJECT_ID_COLUMN, TARGET_COLUMN, TEXT_COLUMN
    
)
from src.data.data_pipeline import data_predict_pipeline
from src.factories import (
    get_callbacks,
    get_dataloaders,
    get_datasets,
    get_lookups,
    get_lr_scheduler,
    get_metric_collections,
    get_model,
    get_optimizer,
    get_text_encoder,
    get_transform,
)
from src.trainer.trainer import Trainer
from src.utils.seed import set_seed

LOGGER = logging.getLogger(name='test')
LOGGER.setLevel(logging.INFO)



def deterministic() -> None:
    """Run experiment deterministically. There will still be some randomness in the backward pass of the model."""
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

    import torch

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)


# Clear the global Hydra instance
GlobalHydra.instance().clear()
#Load configuration

initialize(config_path="../../configs")
#caml multi_res_conv vanillaconv vanillagru laat plm_icd
cfg = compose(config_name="config",
              overrides=["experiment=mimiciv_icd10/plm_icd.yaml",
                         "callbacks=no_wandb",
                         "load_model=null","trainer.epochs=0"]
                        )

best_runs = {
            "CAML": "./experiments/jkz0ogtz",
            "PLMICD": "./experiments/jfywyfod",
            "LAAT": "./experiments/gsycv1tg",
            "MultiResCNN": "./experiments/t1dbhfub",
            "VanillaRNN": "./experiments/mjukdu9c",
            "VanillaConv": "./experiments/vy559dlj",
            }


cfg.load_model = best_runs[cfg.model.name]


if cfg.deterministic:
    deterministic()
else:
    import torch

    if torch.cuda.is_available():
        print("GPU is available")
        print(f"GPU name: {torch.cuda.get_device_name(0)}")
    else:
        print("GPU is not available")
set_seed(cfg.seed)


# Check if CUDA_VISIBLE_DEVICES is set
if "CUDA_VISIBLE_DEVICES" not in os.environ:
    if cfg.gpu != -1 and cfg.gpu is not None and cfg.gpu != "":
        os.environ["CUDA_VISIBLE_DEVICES"] = (
            ",".join([str(gpu) for gpu in cfg.gpu])
            if isinstance(cfg.gpu, list)
            else str(cfg.gpu)
        )
        
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = ""
torch.set_num_threads(os.cpu_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/tmp/ipykernel_9670/1880661831.py:65: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../../configs")


GPU is not available


In [3]:
# mapping Definition
download_dir = Path(DOWNLOAD_DIRECTORY_MIMICIV)

d_icd_procedures = load_gz_file_into_df(
    download_dir / "hosp/d_icd_procedures.csv.gz", dtype={"icd_code": str}
)
d_icd_procedures=d_icd_procedures[d_icd_procedures['icd_version']==10]
d_icd_diagnoses = load_gz_file_into_df(
    download_dir / "hosp/d_icd_diagnoses.csv.gz", dtype={"icd_code": str}
)
d_icd_diagnoses=d_icd_diagnoses[d_icd_diagnoses['icd_version']==10]
d_icd_diagnoses = d_icd_diagnoses[['icd_code', 'long_title']].set_index('icd_code')['long_title'].to_dict()
d_icd_procedures = d_icd_procedures[['icd_code', 'long_title']].set_index('icd_code')['long_title'].to_dict()
d_icd={**d_icd_diagnoses,**d_icd_procedures}

In [4]:
#retrieval
retrieve = pd.read_feather(best_runs[cfg.model.name] + '/retrieve.feather')
retrieve = torch.tensor(retrieve.values).to(device)
print('load retrieve finish')

load retrieve finish


In [5]:
# Specify the path to your JSON file
file_path = best_runs[cfg.model.name] +'/target2index.json'

# Open the file and load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)

token2index = {token: index for index, token in data.items()}

In [ ]:
# Define a Pydantic model for the data structure

class TestDataModel(BaseModel):
    id: List[int]
    text: List[str]
    target: List[List[str]]
    split: str
    Task:str
    iteration: int = Field(2, ge=0, le=10, description="iteration must be between 0 and 10") 
    w_alpha: float = Field(1, ge=0, le=1, description="w_alpha must be between 0 and 1")
    w_beta: float = Field(0.1, ge=0, le=1, description="w_beta must be between 0 and 1")
    AvgTopR: int = Field(10, ge=1, le=50, description="AvgTopR must be between 1 and 50")
    w_gramma: float = Field(0.1, ge=0, le=1, description="w_gramma must be between 0 and 1")
    AvgLowR: int = Field(10, ge=1, le=50, description="AvgLowR must be between 1 and 50")
    Precisionk: int = Field(5, ge=1, le=15, description="precision@k must be between 1 and 15")

# Apply nest_asyncio to allow FastAPI to run within Jupyter Notebook
nest_asyncio.apply()

# Define the FastAPI app
app = FastAPI()

@app.get("/mapping/")
async def mapping():
    return {
        "status": "load mapping successfully",
        "data": {
            values: d_icd[values.replace(".", "")] 
                           for values in token2index.values() 
        }
    }


# Create a POST route for receiving and validating the data structure
@app.post("/predict/")
async def predict(Testdata: TestDataModel):
    
    data = Testdata.model_dump(by_alias=True)
    data = pd.DataFrame(data).rename(columns={'id': ID_COLUMN})
    
    #preprocesss
    preprocessor = TextPreprocessor(
                lower=True,
                remove_special_characters_mullenbach=True,
                remove_special_characters=False,
                remove_digits=True,
                remove_accents=False,
                remove_brackets=False,
                convert_danish_characters=False,
            )

    data=preprocess_documents(df=data, preprocessor=preprocessor)
    

    
    # Here, the validated data can be processed as needed
    data = data_predict_pipeline(config=cfg.data,data=data)

    text_encoder = get_text_encoder(
        config=cfg.text_encoder, data_dir=cfg.data.dir, texts=data.get_train_documents
    )
    label_transform = get_transform(
        config=cfg.label_transform,
        targets=data.all_targets,
        load_transform_path=cfg.load_model,
    )
    text_transform = get_transform(
        config=cfg.text_transform,
        texts=data.get_train_documents,
        text_encoder=text_encoder,
        load_transform_path=cfg.load_model,
    )
    
    data.truncate_text(cfg.data.max_length)
    data.transform_text(text_transform.batch_transform)
    lookups = get_lookups(
        config=cfg.lookup,
        data=data,
        label_transform=label_transform,
        text_transform=text_transform,
    )
   
    model = get_model(
        config=cfg.model, data_info=lookups.data_info, text_encoder=text_encoder
    ).to(device)
    
    datasets = get_datasets(
        config=cfg.dataset,
        data=data,
        text_transform=text_transform,
        label_transform=label_transform,
        lookups=lookups,
    )
    
    dataloaders = get_dataloaders(config=cfg.dataloader, datasets_dict=datasets)
    
    optimizer = get_optimizer(config=cfg.optimizer, model=model)
    accumulate_grad_batches = int(
        max(cfg.dataloader.batch_size / cfg.dataloader.max_batch_size, 1)
    )
    
    num_training_steps = (
        math.ceil(len(dataloaders["train"]) / accumulate_grad_batches)
        * cfg.trainer.epochs
    )
    
    lr_scheduler = get_lr_scheduler(
        config=cfg.lr_scheduler,
        optimizer=optimizer,
        num_training_steps=num_training_steps,
    )
    metric_collections = get_metric_collections(
        config=cfg.metrics,
        number_of_classes=lookups.data_info["num_classes"],
        code_system2code_indices=lookups.code_system2code_indices, # รวมทั้งหมดที่เป็น label diag และ label proc
        split2code_indices=lookups.split2code_indices, # label classs แต่ละ กกลุ่มที่แบ่งไป ตาม 
    )
    callbacks = get_callbacks(config=cfg.callbacks)


    pred = Trainer(
        config=cfg,
        data=data,
        model=model,
        optimizer=optimizer,
        dataloaders=dataloaders,
        metric_collections=metric_collections, # metric มี 4 กลุ่ม ตอน ใช้ metric จะแยกกล่มใครกล่มมั่นไปแล้ว
        callbacks=callbacks,
        lr_scheduler=lr_scheduler,
        lookups=lookups,
        accumulate_grad_batches=accumulate_grad_batches,
        experiment_path = Path(cfg.load_model) # ไว้สำหรับ eval 
    ).to(device)

    if cfg.load_model:
        pred.experiment_path = Path(cfg.load_model)
    predict=pred.fit(predict=True)
    
    ids, logits, targets = predict["ids"], predict["logits"], predict["targets"]



    Task=Testdata.Task
    w_alpha=Testdata.w_alpha
    w_beta=Testdata.w_beta
    w_gramma=Testdata.w_gramma
    AvgTopR=Testdata.AvgTopR
    AvgLowR=Testdata.AvgLowR
    Precisionk = Testdata.Precisionk
    for i in range(1,Testdata.iteration):

        logits=prf(retrieve, logits, AvgTopR=AvgTopR, AvgLowR=AvgLowR, 
                                                w_alpha=w_alpha, w_beta=w_beta, w_gramma=w_gramma, 
                                                chunk_size_b=80000)
 

  

    predict = {id_: {"logits": logit, "targets": target} for id_, logit, target in zip(ids, logits, targets)}

    if Task == "Ranking":
        result = {
            ids.item(): {
                "id": ids.item(),
                "predictions_topk": {
                    token2index[idx]: f'{d_icd[token2index[idx].replace(".", "")]} ({prob})'

                    for idx, prob in zip(
                        logits["logits"].topk(Precisionk).indices.tolist(),
                        logits["logits"].topk(Precisionk).values.tolist()
                    )
                },
                #"target": [token2index[idx] for idx in (data_entry["target"] == 1).nonzero(as_tuple=True)[0].tolist()],
                "match_percentage_topk": round(
                sum(1 for idx in logits["logits"].topk(Precisionk).indices.tolist() if token2index[idx] in [
                    token2index[idx] for idx in (logits["targets"] == 1).nonzero(as_tuple=True)[0].tolist()
                ]) / Precisionk * 100, 2),
            }
            for ids, logits in predict.items()
        }
    else:
        result = {
            ids.item(): {
                "id": ids.item(),
                "classification": {
                    token2index[idx]: f'{d_icd[token2index[idx].replace(".", "")]} ({prob})'
                    for idx, prob in zip(
                        (logits["logits"] > pred.best_db).nonzero(as_tuple=True)[0].tolist(),
                        logits["logits"][logits["logits"] > pred.best_db].tolist()
                    )
                },
                #"target": [token2index[idx] for idx in (data_entry["target"] == 1).nonzero(as_tuple=True)[0].tolist()],
                "match_percentage_classification": round(
                sum(1 for idx in (logits["logits"] > pred.best_db).nonzero(as_tuple=True)[0].tolist() if token2index[idx] in [
                    token2index[idx] for idx in (logits["targets"] == 1).nonzero(as_tuple=True)[0].tolist()
                ]) / len((logits["logits"] > pred.best_db).nonzero(as_tuple=True)[0].tolist()) * 100, 2),

            }
            for ids, logits in predict.items()
        }
    
    del predict,pred,metric_collections,lr_scheduler,num_training_steps,accumulate_grad_batches,optimizer,dataloaders,datasets,callbacks,data,text_encoder,label_transform,text_transform,lookups,model
    gc.collect()
    return {"status": "Predict successfully", "data": result}

# Run the app
config = Config(app=app, host="0.0.0.0", port=8081)
server = Server(config)
server.run()

INFO:     Started server process [9670]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8081 (Press CTRL+C to quit)


INFO:     115.87.76.57:62358 - "POST /predict HTTP/1.1" 307 Temporary Redirect
code_system2code_counts already exists
loaded transform
./files/hfs/RoBERTa-base-PM-M3-Voc-hf
loaded transform


Transforming text...: 100%|██████████| 1/1 [00:00<00:00, 2853.27it/s]
Creating examples train: 0it [00:00, ?it/s]
Creating examples val: 0it [00:00, ?it/s]
Creating examples test: 100%|██████████| 2/2 [00:00<00:00, 636.46it/s]


'Train batch size: 1'

'Accumulating gradients over 16 batch(es).'

...

PosixPath('experiments/jfywyfod')

'Loaded checkpoint from experiments/jfywyfod/best_model.pt'

Loaded best model


Epoch: 19 | Validating on test: 100%|██████████| 1/1 [00:00<00:00,  2.46it/s]


INFO:     127.0.0.1:42284 - "POST /predict/ HTTP/1.1" 200 OK
